In [ ]:
# !pip install dotenv openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.3/730.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16/16 [openai];5;237m━━ 15/16 [openai]c]


In [10]:
import json
import dotenv
import os
import openai
from datetime import datetime

In [14]:
pwd

'/home/hk-user/Works/gitpush/pja_MLOps/Task_recommend/seong_agent'

In [24]:
from pathlib import Path

BASE_DIR = Path.cwd()
JSON_FILE_PATH = BASE_DIR / "data" / "final_normalized_dummy.json"
RCMD_PROMPT_PATH = BASE_DIR / "prompts" / "task_recommend.md"


In [31]:
print(BASE_DIR)

/home/hk-user/Works/gitpush/pja_MLOps/Task_recommend/seong_agent


In [25]:
dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 

In [26]:
# json type check
with open(JSON_FILE_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)
    print(type(data))

<class 'list'>


In [28]:
# workspaceId로 조회
WORKSPACE_ID = 99

with open(JSON_FILE_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)

def extract_workspace_view(id):
    if isinstance(data, list):
        filtered = [ws for ws in data if ws.get("workspaceId") == id]
        filtered = filtered[0] if filtered else None
    else:
        print("** data type is not list **")

    if filtered:
        # print(json.dumps(filtered, indent=4, ensure_ascii=False))
        return filtered
    else:
        print(f"workspaceId={WORKSPACE_ID}에 해당하는 workspace가 없습니다.")

extract_workspace_view(WORKSPACE_ID)

{'workspaceId': 99,
 'recommendedCategories': [{'name': '데이터 수집 및 분석',
   'startDate': '2025-01-01T00:00:00',
   'endDate': '2025-01-31T23:59:59',
   'importance': 5,
   'features': [{'name': '사용자 데이터 수집',
     'startDate': '2025-01-02T00:00:00',
     'endDate': '2025-01-10T23:59:59',
     'importance': 4,
     'actions': [{'name': '스마트폰 앱 개발',
       'startDate': '2025-01-02T00:00:00',
       'endDate': '2025-01-05T23:59:59',
       'importance': 4},
      {'name': '웨어러블 기기와 연동',
       'startDate': '2025-01-06T00:00:00',
       'endDate': '2025-01-08T23:59:59',
       'importance': 3},
      {'name': '데이터 저장소 설계',
       'startDate': '2025-01-09T00:00:00',
       'endDate': '2025-01-10T23:59:59',
       'importance': 3}]},
    {'name': '데이터 분석 알고리즘 개발',
     'startDate': '2025-01-11T00:00:00',
     'endDate': '2025-01-20T23:59:59',
     'importance': 5,
     'actions': [{'name': 'AI 모델 학습',
       'startDate': '2025-01-11T00:00:00',
       'endDate': '2025-01-15T23:59:59',
       'im

In [29]:
##### test - Read task_recommend prompt #####
with open(RCMD_PROMPT_PATH, "r", encoding="utf-8", errors="replace") as f:
    SYSTEM_PROMPT = f.read()
    print("task recommend prompt:\n", SYSTEM_PROMPT)  # test

task recommend prompt:
 # 작업 흐름 기반 액션 추천 프롬프트

## 당신의 역할

당신의 역할은 다음과 같습니다:
- 현재까지 수행된 또는 예정된 작업 흐름을 분석하고,
- 그 흐름에서 자연스럽게 이어질 다음 액션(action)을 **정확히 3가지** 제안하는 것입니다.
- 추천하는 액션은 추상적이거나 모호하지 않아야 하며,
- 구체적으로 어떤 기능에 필요한 어떤 동작을 구현해야 하는지 분명하게 명시해야 합니다.

## 중요 지침 (반드시 따르세요)

- 모든 recommendedCategories와 features에 대해 각각 반드시 **3개 액션을 추천해야 하며**, 1개나 2개만 추천하면 실패로 간주됩니다.
- 모든 추천 액션은 **기존 feature의 actions 배열에 추가되어야 하며**, feature의 name도 명시해야 합니다.
- 액션 각각에 대해 다음 정보를 반드시 포함하세요: name, startDate, endDate, importance
- 출력은 순수 JSON 구조만 있어야 하며, 부가적인 설명, 안내 문구는 포함하지 마세요.
- 출력하는 actions는 추천하는 actions만 최종 출력해야 합니다.

---

## 요구사항

- 반드시 맥락 기반으로 전체 작업 흐름을 분석할 것
- 현실적이고 실현 가능한 다음 작업을 3가지 추천할 것
- 추천하는 작업은 프로젝트 구현에 반드시 필요한 기능이어야 함
- 작업 소요 시간은 기존 작업들의 평균 기간 및 연속성 흐름을 반영하여 startDate, endDate를 합리적으로 지정할 것
- 추천하는 작업의 중요도(importance)는 프로젝트 목표 및 기존 작업과의 연관성을 기반으로 판단할 것
- 추천한 작업은 기존 JSON 구조의 어느 feature 내의 actions 배열에 추가되어야 하는지 명확하게 지정할 것
- actions의 최종 출력은 추천하는 작업(actions)만 출력할 것

### 응답 형식

{
  "workspaceId": "작업 중인 workspa

In [30]:
WORKSPACE_ID = 99  # 유저가 속한 workspaceId

client = openai.OpenAI()
MODEL_ID = 'gpt-4o-mini'

now = datetime.now()
formatted_time = now.strftime("%Y-%m-%dT%H:%M:%S")  # 날짜 형식 지정

def recommend_task(id):

    with open(RCMD_PROMPT_PATH, "r", encoding="utf-8", errors="replace") as f:
        RCMD_PROMPT = f.read()

    # 현재 소속되어 있는 워크스페이스 전체 작업 내용 read
    content = extract_workspace_view(id)
    
    # task recommend prompt -> 프로젝트 진행 단계(맥락) 파악해서 추가해야 할 작업 추천
    prompt = f"""
    다음은 프로젝트 작업 리스트에 대한 정보입니다.
    이 정보를 바탕으로 개발자가 **추가해야 할 새로운 작업(actions)을 3가지 이상** 생성해 주세요.
    최종 출력에서 actions 내부에는 기존의 actions는 제외하고 새로 추가된 actions만 작성합니다.
    추가될 작업은 actions 키 내부에 작성해야 합니다.

    {content}
    {RCMD_PROMPT}

    ※ 출력 포맷은 아래 구조 프롬프트를 정확히 따르며, 설명이나 요약 없이 순수 JSON만 출력하세요.
    """

    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[
            {"role": "system", "content": RCMD_PROMPT},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )
    # print(response.model_dump()["model"])  # model response check

    rcmd_task = response.choices[0].message.content.strip()

    # # Save to json file
    # timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    # filepath = "C:\\Users\\user\\Documents\\pja_MLOps\\Task_recommend\\recommended_list\\"
    # filename = filepath + f"new_task_{timestamp}.json"

    # if isinstance(rcmd_task, str):
    #     rcmd_task = json.loads(rcmd_task)

    # try:
    #     with open(filename, "w", encoding="utf-8") as f:
    #         json.dump(rcmd_task, f, ensure_ascii=False, indent=4)
    #     print(f"** File saved to \"{filename}\" **")
    # except Exception as save_error:
    #     print(f"** Failed to save file: {save_error} **")

    return rcmd_task

print(recommend_task(WORKSPACE_ID))

{
  "workspaceId": 99,
  "recommendedCategories": [
    {
      "name": "데이터 수집 및 분석",
      "startDate": "2025-01-01T00:00:00",
      "endDate": "2025-01-31T23:59:59",
      "importance": 5,
      "features": [
        {
          "name": "사용자 데이터 수집",
          "startDate": "2025-01-02T00:00:00",
          "endDate": "2025-01-10T23:59:59",
          "importance": 4,
          "actions": [
            {
              "name": "사용자 데이터 수집 테스트",
              "startDate": "2025-01-11T00:00:00",
              "endDate": "2025-01-12T23:59:59",
              "importance": 4
            },
            {
              "name": "데이터 수집 결과 분석",
              "startDate": "2025-01-13T00:00:00",
              "endDate": "2025-01-15T23:59:59",
              "importance": 5
            },
            {
              "name": "데이터 수집 최적화 방안 마련",
              "startDate": "2025-01-16T00:00:00",
              "endDate": "2025-01-20T23:59:59",
              "importance": 4
            }
          ]
    